# 2024 第一次数学建模培训作业

01：安装必要的库

In [ ]:
%pip install numpy
%pip install pandas
%pip install scikit-learn
%pip install sklearn
%pip install matplotlib

02：导入所需的库文件

In [12]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import set_config
import warnings
from sklearn.pipeline import Pipeline

03：去除报错提示

In [5]:
warnings.simplefilter('ignore', FutureWarning)

04：导入数据

> 加载和准备葡萄酒质量评估数据

In [7]:
# 1. 数据
table_b11 = pd.DataFrame({
  'Clarity': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0.8, 0.7, 1, 0.9, 1, 1, 1, 0.9, 0.9, 1, 0.7, 0.7, 1, 1, 1, 1, 1, 1, 1, 0.8, 1, 1, 0.8, 0.8, 0.8, 0.8],
  'Aroma': [3.3, 4.4, 3.9, 3.9, 5.6, 4.6, 4.8, 5.3, 4.3, 4.3, 5.1, 3.3, 5.9, 7.7, 7.1, 5.5, 6.3, 5, 4.6, 3.4, 6.4, 5.5, 4.7, 4.1, 6, 4.3, 3.9, 5.1, 3.9, 4.5, 5.2, 4.2, 3.3, 6.8, 5, 3.5, 4.3, 5.2],
  'Body': [2.8, 4.9, 5.3, 2.6, 5.1, 4.7, 4.8, 4.5, 4.3, 3.9, 4.3, 5.4, 5.7, 6.6, 4.4, 5.6, 5.4, 5.5, 4.1, 5, 5.4, 5.3, 4.1, 4, 5.4, 4.6, 4, 4.9, 4.4, 3.7, 4.3, 3.8, 3.5, 5, 5.7, 4.7, 5.5, 4.8],
  'Flavor': [3.1, 3.5, 4.8, 3.1, 5.5, 5, 4.8, 4.3, 3.9, 4.7, 4.5, 4.3, 7, 6.7, 5.8, 5.6, 4.8, 5.5, 4.3, 3.4, 6.6, 5.3, 5, 4.1, 5.7, 4.7, 5.1, 5, 5, 2.9, 5, 3, 4.3, 6, 5.5, 4.2, 3.5, 5.7],
  'Oakiness': [4.1, 3.9, 4.7, 3.6, 5.1, 4.1, 3.3, 5.2, 2.9, 3.9, 3.6, 3.6, 4.1, 3.7, 4.1, 4.4, 4.6, 4.1, 3.1, 3.4, 4.8, 3.8, 3.7, 4, 4.7, 4.9, 5.1, 5.1, 4.4, 3.9, 6, 4.7, 4.5, 5.2, 4.8, 3.3, 5.8, 3.5],
  'Quality': [9.8, 12.6, 11.9, 11.1, 13.3, 12.8, 12.8, 12, 13.6, 13.9, 14.4, 12.3, 16.1, 16.1, 15.5, 15.5, 13.8, 13.8, 11.3, 7.9, 15.1, 13.5, 10.8, 9.5, 12.7, 11.6, 11.7, 11.9, 10.8, 8.5, 10.7, 9.1, 12.1, 14.9, 13.5, 12.2, 10.3, 13.2]
})

建立一个完整的线性回归模型以预测葡萄酒质量的:

1. 从表格数据中选择特征列('Clarity', 'Aroma', 'Body', 'Flavor', 'Oakiness')作为模型输入X

2. 选择标签列'Quality'作为需要预测的目标变量y

3. 使用train_test_split函数对数据进行训练集测试集划分,获得X_train, X_test, y_train, y_test

4. 构建线性回归模型 LinearRegression,并基于训练集数据 fit模型求参数

5. 在测试集上使用fit好的模型lr进行预测,计算预测值和真实值

6. 计算测试集上预测值和真实值的R^2分数,评估完整模型的预测性能

主要功能是:

- 从原数据中提取特征和标签变量
- 划分训练测试集
- 构建完整线性回归模型进行预测 
- 计算测试集R^2评价模型效果

这种建模过程中包含数据准备、模型构建、模型评价三个关键步骤,即标准的机器学习工作流程。可以对模型在全部数据上的表现进行评价。

In [9]:
# 2. 建立完整模型
X = table_b11[['Clarity', 'Aroma', 'Body', 'Flavor', 'Oakiness']]
y = table_b11['Quality']
X_train, X_test, y_train, y_test = train_test_split(X, y)

lr = LinearRegression().fit(X_train, y_train)
print(f'完整模型R^2为:{r2_score(y_test,lr.predict(X_test))}')

完整模型R^2为:0.4029095063296203


这段代码是使用逐步选择(RFE)方法,从所有特征变量中选择top3重要变量:

1. 使用RFE类,传入线性回归模型lr和需要选择的特征数n_features_to_select=3

2. 调用RFE对象的fit方法,将训练数据X_train和y_train传入,进行特征选择操作

3. RFE内部会重复的拟合线性回归模型,每次移除影响最小的一个特征。选择能够最大限度提升模型性能的前3个特征。

4. 通过rfe.support_属性可以获取被选中(支持)的特征下标

5. 将支持下标映射为真实的特征名,打印出最终被选择的top3变量

主要过程:

- 构建RFE对象,设置线性回归模型和需要选择的特征数
- 使用训练集数据进行逐步选择,重复选择保留影响最大的特征  
- 输出选择结果,就是该模型中最重要的前3个影响因素

RFE使用封装的方式实现了线性模型中的特征选择,可以很容易的选择出特征大集合中对目标影响最大的几个变量。

In [10]:
# 3. 逐步选择
rfe = RFE(lr,n_features_to_select=3).fit(X_train, y_train)
print(f'选出的变量为:{X.columns[rfe.support_]}\n')

选出的变量为:Index(['Clarity', 'Body', 'Flavor'], dtype='object')



这段代码是用来进行新的样本预测及置信区间计算的:

1. 定义一个新的样本x_new,包含5个已选择特征的测试值

2. 使用步骤2训练好的线性回归模型lr,对x_new进行预测:

   - lr.predict()函数生成新的样本的预测值pred

3. 计算预测值的置信区间:

   - lr.coef_获取线性回归系数
   - np.std()计算系数的标准差
   - 预测值乘以1.96倍标准差,即95%置信区间范围

4. 打印预测值pred和置信区间conf_interval

主要功能:

- 定义新的预测样本
- 使用已训练模型进行预测 
- 计算预测值的置信区间范围
- 预测结果附加置信度信息

通过这个过程可以在已有模型基础上,对新的特征样本进行预测,同时给出预测值的可信程度范围,为预测结果提供参考依据。

In [11]:
# 4. 预测和区间
x_new = pd.DataFrame({'Clarity': [1.1], 'Aroma': [5.1], 'Body': [5.6], 'Flavor': [5.5], 'Oakiness': [14]})
pred = lr.predict(x_new)
conf_interval = lr.predict(x_new) * 1.96 * np.std(lr.coef_) # 置信区间
print(f'预测值:{pred}')
print(f'置信区间:{conf_interval}')

预测值:[5.95165779]
置信区间:[15.29527134]
